# 5.0 Leave-One-Study-Out Evaluation (Z-Score)

**模型架构**: Z-Score (Only Species, Z-sorted, No Value/Condition)
**任务**: 对 6 个目标 Study 进行留一法交叉验证 (LOO-CV) 和零样本迁移 (Zero-shot) 测试。

**流程**：
1. **Zero-shot**: 使用 Group A 微调好的模型，直接预测 6 个 Study。
2. **LOO Fine-tuning**: 轮流选择 1 个 Study 作为测试集，其余 5 个作为训练集，微调模型并评估。

In [1]:
import os
import torch
import numpy as np
import pandas as pd
import json
import joblib
import gc
import shutil
from argparse import Namespace
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from torch.utils.data import Subset
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# --- v2.0 自定义模块 ---
from MiCoGPT.utils_vCross.model_vCross import MiCoGPTConfig, MiCoGPTForSequenceClassification
from MiCoGPT.utils_vCross.collator_vCross import MiCoGPTClassificationCollator
from MiCoGPT.utils_vCross.corpus_zscore import MiCoGPTCorpusZScore

# --- 复用工具函数 ---
from MiCoGPT.utils.finetune_v2 import SubsetWithLabels
from MiCoGPT.utils.mgm_utils import eval_and_save

import warnings
warnings.filterwarnings("ignore")

In [2]:
args = Namespace(
    # 语料库 (Z-Score)
    input="../data/vCross/ResMicroDB_90338_vCross_ZScore.pkl",
    
    # 基础模型 (Group A 微调后的模型，用于 Zero-shot 和作为 LOO 的初始权重)
    # 您也可以改为使用预训练模型 "../models/pretrain_vCross_zscore"
    base_model="../models/finetuned_vCross_zscore_valLoss",
    
    # 标签编码器 (必须使用 Group A 的 Encoder 以保持标签一致)
    label_encoder_path="../models/finetuned_vCross_zscore_valLoss/label_encoder.joblib",
    
    # 输出根目录
    output_root="../models/LOO_vCross_zscore_valLoss",
    
    # 目标 Studies
    target_studies = [
        'PRJNA439311', 'PRJNA282010', 'PRJNA296567', 
        'PRJNA632472', 'PRJNA1108737', 'PRJNA820972'
    ],
    
    label_col="Is_Healthy",
)

os.makedirs(args.output_root, exist_ok=True)

In [3]:
print("Loading Corpus...")
with open(args.input, "rb") as f:
    import pickle
    corpus = pickle.load(f)

print("Loading Label Encoder...")
le = joblib.load(args.label_encoder_path)

# 兼容 LabelEncoder 和 OneHotEncoder
if hasattr(le, "classes_"):
    classes = le.classes_
else:
    classes = le.categories_[0]
print(f"Classes: {classes}")

# 筛选 Split_Group == 'C' 且在目标 Studies 中的样本
meta = corpus.metadata
mask = (
    (meta["Split_Group"] == "C") & 
    (meta["Project_ID"].isin(args.target_studies)) & 
    (meta[args.label_col].notna())
)
target_indices = np.where(mask)[0]
target_subset = Subset(corpus, target_indices)
print(f"Selected {len(target_subset)} samples from {len(args.target_studies)} studies.")

Loading Corpus...
Loading Label Encoder...
Classes: [False True]
Selected 1029 samples from 6 studies.


In [4]:
# 定义 Z-Score 专用的 Collator (屏蔽 Value/Condition)
class BaselineClassificationCollator(MiCoGPTClassificationCollator):
    def __call__(self, examples):
        batch = super().__call__(examples)
        batch["value_ids"] = None
        batch["condition_ids"] = None
        return batch

collator = BaselineClassificationCollator(corpus.tokenizer, max_length=512)

In [5]:
def get_study_subset(study_id, indices_map):
    # indices_map: global_index -> local_index_in_subset
    # 这里我们需要从 corpus 中提取特定 study 的 indices
    study_mask = (meta["Project_ID"] == study_id) & mask # mask is the global filter
    study_indices = np.where(study_mask)[0]
    return Subset(corpus, study_indices)

def prepare_dataset(subset, le):
    # 提取标签并编码
    indices = subset.indices
    labels_raw = corpus.metadata.iloc[indices][args.label_col].values
    
    # 注意：这里不再进行 astype(str) 转换，而是直接使用原始数据
    # 以保持与训练时 prepare_labels_for_subset 的行为一致
    
    if hasattr(le, "classes_"):
        labels_encoded = le.transform(labels_raw)
    else:
        # OneHotEncoder
        labels_onehot = le.transform(labels_raw.reshape(-1, 1))
        if hasattr(labels_onehot, "toarray"):
            labels_onehot = labels_onehot.toarray()
        labels_encoded = np.argmax(labels_onehot, axis=1)
    
    return SubsetWithLabels(subset, torch.tensor(labels_encoded, dtype=torch.long))

## 任务一：Zero-shot Evaluation
加载 Group A 模型，直接在 6 个 Study 上预测。

In [6]:
print("=== Starting Zero-shot Evaluation ===")

# 加载模型
model = MiCoGPTForSequenceClassification.from_pretrained(
    args.base_model, 
    num_labels=len(classes),
    ignore_mismatched_sizes=True
)

trainer = Trainer(
    model=model,
    data_collator=collator,
    # 显存优化
    args=TrainingArguments(output_dir=".tmp", per_device_eval_batch_size=32, eval_accumulation_steps=None)
)

for study in args.target_studies:
    print(f"Evaluating on {study}...")
    study_subset = get_study_subset(study, None)
    if len(study_subset) == 0:
        print(f"Warning: No samples found for {study}")
        continue
        
    test_ds = prepare_dataset(study_subset, le)
    
    # 预测
    predictions = trainer.predict(test_ds)
    y_score = predictions.predictions
    y_true = predictions.label_ids
    
    # 保存
    save_dir = f"{args.output_root}/zero_shot/{study}"
    os.makedirs(save_dir, exist_ok=True)
    
    eval_and_save(
        y_score=y_score,
        y_true=y_true,
        label_names=[str(c) for c in classes],
        save_dir=save_dir,
        activation="softmax"
    )
    print(f"Saved results to {save_dir}")

del model, trainer
gc.collect()
torch.cuda.empty_cache()

=== Starting Zero-shot Evaluation ===
Evaluating on PRJNA439311...


Evaluating biome source: False
       TN  FP  FN  TP     Acc      Sn      Sp     TPR     FPR      Rc      Pr  \
t                                                                               
0.000   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.001   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.002   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.003   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.004   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
...    ..  ..  ..  ..     ...     ...     ...     ...     ...     ...     ...   
0.997   2  14   9  77  0.7745  0.8953  0.1250  0.8953  0.8750  0.8953  0.8462   
0.998   2  14  13  73  0.7353  0.8488  0.1250  0.8488  0.8750  0.8488  0.8391   
0.999   7   9  26  60  0.6569  0.6977  0.4375  0.6977  0.5625  0.6977  0.8696   
1.000  16   0  86   0  0.1569  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000 

Evaluating biome source: False
       TN  FP  FN  TP     Acc      Sn      Sp     TPR     FPR      Rc      Pr  \
t                                                                               
0.000   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.001   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.002   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.003   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.004   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
...    ..  ..  ..  ..     ...     ...     ...     ...     ...     ...     ...   
0.997   8  11   1  17  0.6757  0.9444  0.4211  0.9444  0.5789  0.9444  0.6071   
0.998  10   9   1  17  0.7297  0.9444  0.5263  0.9444  0.4737  0.9444  0.6538   
0.999  12   7   3  15  0.7297  0.8333  0.6316  0.8333  0.3684  0.8333  0.6818   
1.000  19   0  18   0  0.5135  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000 

Evaluating biome source: False
       TN  FP  FN  TP     Acc    Sn      Sp   TPR     FPR    Rc      Pr  \
t                                                                         
0.000   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
0.001   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
0.002   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
0.003   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
0.004   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
...    ..  ..  ..  ..     ...   ...     ...   ...     ...   ...     ...   
0.997  11   3  16   4  0.4412  0.20  0.7857  0.20  0.2143  0.20  0.5714   
0.998  11   3  17   3  0.4118  0.15  0.7857  0.15  0.2143  0.15  0.5000   
0.999  13   1  18   2  0.4412  0.10  0.9286  0.10  0.0714  0.10  0.6667   
1.000  14   0  20   0  0.4118  0.00  1.0000  0.00  0.0000  0.00  0.0000   
1.001  14   0  20   0  0.4118  0.00  1.0000  0.00  0.0000  0.00  0.00

Evaluating biome source: False
        TN   FP   FN   TP     Acc      Sn      Sp     TPR     FPR      Rc  \
t                                                                           
0.000    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.001    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.002    1  206    0  338  0.6220  1.0000  0.0048  1.0000  0.9952  1.0000   
0.003    1  206    0  338  0.6220  1.0000  0.0048  1.0000  0.9952  1.0000   
0.004    1  206    0  338  0.6220  1.0000  0.0048  1.0000  0.9952  1.0000   
...    ...  ...  ...  ...     ...     ...     ...     ...     ...     ...   
0.997  154   53  173  165  0.5853  0.4882  0.7440  0.4882  0.2560  0.4882   
0.998  164   43  194  144  0.5651  0.4260  0.7923  0.4260  0.2077  0.4260   
0.999  179   28  234  104  0.5193  0.3077  0.8647  0.3077  0.1353  0.3077   
1.000  207    0  338    0  0.3798  0.0000  1.0000  0.0000  0.0000  0.0000   
1.001  207    0  338    0  0.3798  0.0000  1.

Evaluating biome source: False
       TN  FP  FN  TP     Acc      Sn   Sp     TPR  FPR      Rc      Pr  \
t                                                                         
0.000   0  76   0  76  0.5000  1.0000  0.0  1.0000  1.0  1.0000  0.5000   
0.001   0  76   1  75  0.4934  0.9868  0.0  0.9868  1.0  0.9868  0.4967   
0.002   0  76   1  75  0.4934  0.9868  0.0  0.9868  1.0  0.9868  0.4967   
0.003   0  76   1  75  0.4934  0.9868  0.0  0.9868  1.0  0.9868  0.4967   
0.004   0  76   1  75  0.4934  0.9868  0.0  0.9868  1.0  0.9868  0.4967   
...    ..  ..  ..  ..     ...     ...  ...     ...  ...     ...     ...   
0.997  76   0  73   3  0.5197  0.0395  1.0  0.0395  0.0  0.0395  1.0000   
0.998  76   0  73   3  0.5197  0.0395  1.0  0.0395  0.0  0.0395  1.0000   
0.999  76   0  75   1  0.5066  0.0132  1.0  0.0132  0.0  0.0132  1.0000   
1.000  76   0  76   0  0.5000  0.0000  1.0  0.0000  0.0  0.0000  0.0000   
1.001  76   0  76   0  0.5000  0.0000  1.0  0.0000  0.0  0.0000  0.00

Evaluating biome source: False
       TN  FP   FN   TP     Acc      Sn      Sp     TPR     FPR      Rc  \
t                                                                         
0.000   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.001   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.002   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.003   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.004   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
...    ..  ..  ...  ...     ...     ...     ...     ...     ...     ...   
0.997  24  30   17   88  0.7044  0.8381  0.4444  0.8381  0.5556  0.8381   
0.998  27  27   21   84  0.6981  0.8000  0.5000  0.8000  0.5000  0.8000   
0.999  31  23   31   74  0.6604  0.7048  0.5741  0.7048  0.4259  0.7048   
1.000  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.0000   
1.001  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.00

## 任务二：Leave-One-Study-Out Fine-tuning
轮流选一个做测试，其余做训练。

In [7]:
import numpy as np
import gc
import torch
from transformers import TrainerCallback

print("=== Starting Nested LOO (Double LOO) Fine-tuning ===")

# 辅助函数：清理显存
def clean_memory():
    gc.collect()
    torch.cuda.empty_cache()

# 辅助函数：计算指标
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

# 定义最大搜索轮数
MAX_EPOCHS = 50

# === 1. 外层循环：留一个作为最终测试集 (Test Study) ===
for test_study in args.target_studies:
    print(f"\n{'='*20}")
    print(f">>> Outer Loop: Test Target = {test_study}")
    print(f"{'='*20}")
    
    # 确定当前的训练池 (5个 studies)
    train_pool_studies = [s for s in args.target_studies if s != test_study]
    
    # 用于存储内层 5 折的 Loss 曲线
    # 结构: [ [fold1_epoch1_loss, fold1_epoch2_loss...], [fold2...], ... ]
    all_folds_val_losses = []
    
    # === 2. 内层循环：5-Fold CV 寻找最佳 Epoch ===
    print(f"\n[Inner Loop] Starting 5-fold CV on train pool to find optimal epochs...")
    
    for fold_idx, val_study in enumerate(train_pool_studies):
        print(f"  Inner Fold {fold_idx+1}/5: Val Study = {val_study}")
        
        # 2.1 准备内层数据
        # 内层训练集：训练池中除了当前验证 Study 之外的 4 个
        inner_train_studies = [s for s in train_pool_studies if s != val_study]
        
        # 构建 Dataset
        inner_train_mask = (meta["Project_ID"].isin(inner_train_studies)) & mask
        inner_train_ds = prepare_dataset(Subset(corpus, np.where(inner_train_mask)[0]), le)
        
        inner_val_subset = get_study_subset(val_study, None)
        inner_val_ds = prepare_dataset(inner_val_subset, le)
        
        # 2.2 初始化模型 (每次都要重置)
        model = MiCoGPTForSequenceClassification.from_pretrained(
            args.base_model, 
            num_labels=len(classes),
            ignore_mismatched_sizes=True
        )
        
        # 2.3 训练配置
        # 注意：内层不需要 load_best_model_at_end，我们需要完整的 loss 曲线
        inner_output_dir = f"{args.output_root}/temp_inner/{test_study}/{val_study}"
        training_args = TrainingArguments(
            output_dir=inner_output_dir,
            num_train_epochs=MAX_EPOCHS,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            learning_rate=1e-5,
            weight_decay=0.01,
            evaluation_strategy="epoch", # 每个 epoch 测一次
            save_strategy="no",          # 内层为了省空间和时间，不保存 checkpoint
            load_best_model_at_end=False,
            logging_steps=10,
            dataloader_num_workers=0,
            report_to="none"             # 避免 wandb 等干扰
        )
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=inner_train_ds,
            eval_dataset=inner_val_ds,
            data_collator=collator,
            compute_metrics=compute_metrics
        )
        
        trainer.train()
        
        # 2.4 提取验证 Loss 曲线
        # log_history 中包含训练 loss 和 eval loss，我们需要筛选出 eval_loss
        history = trainer.state.log_history
        # 提取包含 'eval_loss' 的记录，并按 epoch 排序
        eval_logs = [log for log in history if 'eval_loss' in log]
        # 确保按 epoch 顺序（虽然通常已经是顺序的）
        eval_logs.sort(key=lambda x: x['epoch'])
        
        fold_losses = [log['eval_loss'] for log in eval_logs]
        
        # 简单校验：如果因为某些原因记录数不够，补齐或截断（通常不会发生）
        if len(fold_losses) < MAX_EPOCHS:
            print(f"    Warning: Expected {MAX_EPOCHS} logs, got {len(fold_losses)}. Padding with infinity.")
            fold_losses += [float('inf')] * (MAX_EPOCHS - len(fold_losses))
        
        all_folds_val_losses.append(fold_losses)
        
        # 清理资源
        del model, trainer, inner_train_ds, inner_val_ds
        clean_memory()
    
    # === 3. 聚合结果，选择最佳 Epoch ===
    # 将 list 转为 numpy array: shape (5, MAX_EPOCHS)
    loss_matrix = np.array(all_folds_val_losses)
    # 计算每个 epoch 的平均 loss
    avg_loss_per_epoch = np.mean(loss_matrix, axis=0)
    
    # 找到 loss 最小的索引 (epoch 从 1 开始，所以 index + 1)
    best_epoch_idx = np.argmin(avg_loss_per_epoch)
    best_epoch = int(best_epoch_idx + 1)
    min_loss = avg_loss_per_epoch[best_epoch_idx]
    
    print(f"\n[Result] Best Epoch for {test_study} is: {best_epoch} (Avg Val Loss: {min_loss:.4f})")
    
    # === 4. Refit: 用最佳 Epoch 在整个训练池上重训 ===
    print(f"[Refit] Retraining on all 5 training studies for {best_epoch} epochs...")
    
    # 4.1 准备 Refit 数据 (所有 5 个训练 study 合并)
    refit_train_mask = (meta["Project_ID"].isin(train_pool_studies)) & mask
    refit_train_ds = prepare_dataset(Subset(corpus, np.where(refit_train_mask)[0]), le)
    
    # 准备最终测试数据
    final_test_subset = get_study_subset(test_study, None)
    final_test_ds = prepare_dataset(final_test_subset, le)
    
    print(f"  Refit Train Samples: {len(refit_train_ds)}")
    print(f"  Final Test Samples:  {len(final_test_ds)}")
    
    # 4.2 初始化模型
    model = MiCoGPTForSequenceClassification.from_pretrained(
        args.base_model, 
        num_labels=len(classes),
        ignore_mismatched_sizes=True
    )
    
    # 4.3 训练配置
    final_output_dir = f"{args.output_root}/finetuned/{test_study}"
    training_args = TrainingArguments(
        output_dir=final_output_dir,
        num_train_epochs=best_epoch,     # <--- 关键：使用选出的最佳 Epoch
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        learning_rate=1e-5,
        weight_decay=0.01,
        evaluation_strategy="no",        # Refit 阶段不需要验证
        save_strategy="no",              # Refit 结束后直接保存或预测，不需要中间存 checkpoint
        logging_steps=10,
        dataloader_num_workers=0
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=refit_train_ds,
        data_collator=collator
    )
    
    trainer.train()
    
    # === 5. 最终预测与保存 ===
    print(f"[Predict] Predicting on {test_study}...")
    predictions = trainer.predict(final_test_ds)
    
    eval_and_save(
        predictions.predictions, 
        predictions.label_ids, 
        [str(c) for c in classes], 
        final_output_dir, 
        activation="softmax"
    )
    
    print(f"Saved results to {final_output_dir}")
    
    # 记录最佳参数信息到文件（可选）
    with open(f"{final_output_dir}/best_params.txt", "w") as f:
        f.write(f"Best Epoch: {best_epoch}\n")
        f.write(f"Avg Val Loss: {min_loss:.4f}\n")
        f.write(f"Loss Curve: {avg_loss_per_epoch.tolist()}\n")
    
    # 清理资源
    del model, trainer, refit_train_ds, final_test_ds
    clean_memory()

print("\n=== All LOO Rounds Completed ===")

=== Starting Nested LOO (Double LOO) Fine-tuning ===

>>> Outer Loop: Test Target = PRJNA439311

[Inner Loop] Starting 5-fold CV on train pool to find optimal epochs...
  Inner Fold 1/5: Val Study = PRJNA282010


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.329600,2.110493,0.486486,0.318428
2,1.022500,1.768742,0.540541,0.428256
3,0.770600,1.528035,0.567568,0.501982
4,0.716400,1.419111,0.567568,0.501982
5,0.829300,1.299313,0.567568,0.501982
6,0.743600,1.213952,0.594595,0.542967
7,0.650000,1.140943,0.594595,0.542967
8,0.607900,1.090168,0.621622,0.581601
9,0.651100,1.066011,0.621622,0.581601
10,0.630700,1.035291,0.621622,0.581601


  Inner Fold 2/5: Val Study = PRJNA296567


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.198900,1.591019,0.588235,0.435730
2,1.084800,1.475236,0.588235,0.435730
3,0.934000,1.400855,0.588235,0.435730
4,0.855500,1.354057,0.588235,0.435730
5,0.704700,1.312773,0.588235,0.435730
6,0.701500,1.275826,0.588235,0.435730
7,0.691200,1.237320,0.588235,0.435730
8,0.653500,1.224389,0.588235,0.435730
9,0.686500,1.212390,0.588235,0.435730
10,0.601000,1.191593,0.588235,0.435730


  Inner Fold 3/5: Val Study = PRJNA632472


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.085300,0.994598,0.552632,0.529069
2,0.890500,0.943883,0.546053,0.541588
3,0.868800,0.922985,0.519737,0.518047
4,0.828300,0.911709,0.519737,0.518716
5,0.745100,0.900027,0.526316,0.526234
6,0.772500,0.890658,0.532895,0.532875
7,0.674800,0.880204,0.532895,0.532875
8,0.667100,0.875365,0.539474,0.539394
9,0.589800,0.871030,0.546053,0.546033
10,0.521900,0.865722,0.539474,0.539394



[Result] Best Epoch for PRJNA820972 is: 16 (Avg Val Loss: 1.0807)
[Refit] Retraining on all 5 training studies for 16 epochs...
  Refit Train Samples: 870
  Final Test Samples:  159


Step,Training Loss
10,1.424000
20,1.102000
30,1.120400
40,0.982600
50,1.025600
60,0.860100
70,0.837400
80,0.814100
90,0.961200
100,0.760700


[Predict] Predicting on PRJNA820972...


Evaluating biome source: False
       TN  FP   FN   TP     Acc      Sn      Sp     TPR     FPR      Rc  \
t                                                                         
0.000   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.001   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.002   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.003   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.004   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
...    ..  ..  ...  ...     ...     ...     ...     ...     ...     ...   
0.997  48   6   79   26  0.4654  0.2476  0.8889  0.2476  0.1111  0.2476   
0.998  53   1   84   21  0.4654  0.2000  0.9815  0.2000  0.0185  0.2000   
0.999  54   0   94   11  0.4088  0.1048  1.0000  0.1048  0.0000  0.1048   
1.000  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.0000   
1.001  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.00